# Exercise Sheet 3 - The QR decomposition

In this exercise we will implement the QR-Algorithm using Householder transformations. Recall from the lecture:

For an arbitrary $v \neq 0$, we define $$P = I - 2 \frac{vv^\top}{v^\top v}.$$ These matrices are called Householder reflectors and are symmetric and orthogonal.

## (1) Householder Reflectors

We want to use Householder reflectors to zero out components in matrices. Let's start with a simple vector. Given an arbitrary vector $x \neq 0$, we want to find a Householder reflector $P$ as above such that $$ P x = k e_1,$$ where $k$ is a constant an $e_1$ the first unit vector. 

**Task: Write a function that takes $x$ as an input and returns the matrix $P$ that projects $x$ to a multiple of the first unit vector.**

Hint: Consult your lecture notes if you don't remember how it works.

In [178]:
import numpy as np
import numpy.linalg as nplin
def h(x):
    import numpy as np
    import numpy.linalg as nplin
    e1=np.eye(len(x))[0]
    y=nplin.norm(x)*e1
    v=x-y
    i=np.identity(len(x))
    h=i-2*np.outer(v, v)/ np.inner(v,v)
    return h

Apply your function to a random vector generated by 
```
x = np.random.rand(5)
```
The result of the matrix vector multiplication may not be **exactly** what we wanted. Why is that?

In [179]:
x = np.random.rand(5)

In [180]:
x

array([0.67325587, 0.47079528, 0.17277084, 0.35140555, 0.49274661])

In [181]:
h(x)

array([[ 0.65053993,  0.45491045,  0.16694147,  0.33954898,  0.47612112],
       [ 0.45491045,  0.40781929, -0.21731644, -0.44200865, -0.61979176],
       [ 0.16694147, -0.21731644,  0.92024996, -0.16220682, -0.22744905],
       [ 0.33954898, -0.44200865, -0.16220682,  0.67008103, -0.46261777],
       [ 0.47612112, -0.61979176, -0.22744905, -0.46261777,  0.3513098 ]])

In [183]:
np.dot(h(x),x).round(5)

array([ 1.03492, -0.     , -0.     ,  0.     , -0.     ])

## (2) Application of Householder Reflectors

The function you wrote for the previous task is already the main tool for computing the QR decomposition. For a given matrix $A\in\mathbb{R}^{m,n}$, we want to compute the orthogonal matrix $Q$ such that $$A = Q R.$$ In the lecture, we discussed how to compute matrices $H_1, H_2, \dots, H_n$ with $Q^\top = H_4 H_3 \cdots  H_1$.

Let's start simple by just using our previous results.

**Task: Define the matrix**
```
M = np.array([[3,2,9],[0,5,6],[4,7,1]])
```
**and use the function from part (1) to project the entries from of first column to a multiple of the first unit vector. Print the matrix $M$ after the application of the projection.**

Remark: The task is not to treat the first column as a vector, but to determine the matrix $H_1$. This means that the other entries in the matrix also change when $H_1$ and $M$ are multiplied.

In [309]:
M = np.array([[3,2,9],[0,5,6],[4,7,1]])
H1=h(M.transpose()[0])
M1= np.matmul( H1,M)

In [310]:
H1

array([[ 0.6,  0. ,  0.8],
       [ 0. ,  1. ,  0. ],
       [ 0.8,  0. , -0.6]])

In [311]:
np.dot( H1,M.transpose()[0])

array([5., 0., 0.])

**Task: Create a zero below the diagonal in the second column, i.e., calculate the matrix $H_2$ and multiply it with your previous result.** 

In [323]:
h2=h(M1.transpose()[1, 1:])
a=np.ones((1, 1))
b=np.zeros((1, len(h2)))
c=np.zeros((len(h2),1))
H2=np.bmat([[a, b],[c,h2]])
H2

matrix([[ 1.        ,  0.        ,  0.        ],
        [ 0.        ,  0.8872168 , -0.46135274],
        [ 0.        , -0.46135274, -0.8872168 ]])

Now, that you have understood the concept, we can write a function that generalizes it to arbitrary $m$ by $n$ matrices.

**Task: Write a function that returns a list with the matrices $H_j \in\mathbb{R}^{m,m}$, $j = 1,2,\dots,n$. The input
is a $m\times n$ matrix $A$.**

Remark: Do not modify the input matrix A! Since $A$ can be rectangular, think about when to stop the algorithm.

In [327]:
def fact(A):
    import copy
    B=A.transpose()
    q1=h(B[0:1,0:])
    B2= np.matmul(q1,A)
    lst=[q1]
    m=np.shape(B)[1]
    n=min(np.shape(A)[0],np.shape(A)[1])
    for i in range(1,n):
        q2=np.eye(np.shape(A)[0])
        B2=np.matmul(q2,B2)
        s=h(B[i:i+1,i:])
        a=np.ones((i, i))
        c=np.zeros((i, m-i))
        b=np.zeros(( m-i,i))
        q2=np.bmat([[a, c],[b,s]])
        lst.append(q2)
    return lst[::-1]

Test your function using the matrix
```
A = np.array([[1,3,8,6,2],[2,5,6,3,4],[4,7,9,5,1], [2,4,6,1,7]])
```
by multiplying the matrices from your function with it!

In [328]:
A = np.array([[1,3,8,6,2],[2,5,6,3,4],[4,7,9,5,1], [2,4,6,1,7]])
fact(A)

<ipython-input-178-53aa0a7d0166>:10: RuntimeWarning: invalid value encountered in true_divide
  h=i-2*np.outer(v, v)/ np.inner(v,v)


[matrix([[ 1.,  1.,  1.,  0.],
         [ 1.,  1.,  1.,  0.],
         [ 1.,  1.,  1.,  0.],
         [ 0.,  0.,  0., nan]]),
 matrix([[ 1.        ,  1.        ,  0.        ,  0.        ],
         [ 1.        ,  1.        ,  0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.75092957,  0.3396177 ],
         [ 0.        ,  0.        ,  0.3396177 , -0.75092957]]),
 matrix([[ 1.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  0.28638106,  0.60447578,  0.12733369],
         [ 0.        ,  0.60447578,  0.78078019,  0.51632357],
         [ 0.        ,  0.12733369,  0.51632357, -0.06716124]]),
 array([[0.08571429, 0.31428571, 0.77142857, 0.31428571],
        [0.31428571, 0.48571429, 0.82857143, 0.48571429],
        [0.77142857, 0.82857143, 0.94285714, 0.82857143],
        [0.31428571, 0.48571429, 0.82857143, 0.48571429]])]

In [330]:
Qt=nplin.multi_dot(fact(A))

<ipython-input-178-53aa0a7d0166>:10: RuntimeWarning: invalid value encountered in true_divide
  h=i-2*np.outer(v, v)/ np.inner(v,v)


In [332]:
q=Qt.transpose()

In [337]:
r=np.matmul(Qt,A)

matrix([[33.75351216, 68.01087121, 97.97133465, 49.66728033, 43.14777959],
        [33.75351216, 68.01087121, 97.97133465, 49.66728033, 43.14777959],
        [33.75351216, 68.01087121, 97.97133465, 49.66728033, 43.14777959],
        [        nan,         nan,         nan,         nan,         nan]])

In [338]:
np.matmul(q,r)

matrix([[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]])

## (3) QR Factorization

Now, we want a complete QR function.

 **Task: Write a function**

```
QR(A)
```

**that returns the orthogonal matrix $Q$ and the upper right triangular matrix $R$. The input is an $m\times n$ matrix $A$.**

In [340]:
def QR(A):
    Qt=nplin.multi_dot(fact(A))
    q=Qt.transpose()
    r=np.matmul(Qt,A)
    return q,r

In [341]:
QR(A)

<ipython-input-178-53aa0a7d0166>:10: RuntimeWarning: invalid value encountered in true_divide
  h=i-2*np.outer(v, v)/ np.inner(v,v)


(matrix([[2.22260513, 2.22260513, 2.22260513,        nan],
         [3.08198426, 3.08198426, 3.08198426,        nan],
         [4.8007425 , 4.8007425 , 4.8007425 ,        nan],
         [3.08198426, 3.08198426, 3.08198426,        nan]]),
 matrix([[33.75351216, 68.01087121, 97.97133465, 49.66728033, 43.14777959],
         [33.75351216, 68.01087121, 97.97133465, 49.66728033, 43.14777959],
         [33.75351216, 68.01087121, 97.97133465, 49.66728033, 43.14777959],
         [        nan,         nan,         nan,         nan,         nan]]))

The numpy library also provides a method to compute the QR factorization of a matrix. Read the documentation [here](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.qr.html) and compare the outputs $Q$ and $R$ of your function with the outputs from the numpy libray. Do you notice any differences? Can you explain the differences?

In [342]:
Q, R = nplin.qr(A)

In [343]:
Q,R

(array([[-0.2       , -0.60448772,  0.74413167, -0.20214514],
        [-0.4       , -0.62773725, -0.65405257, -0.13476342],
        [-0.8       ,  0.48824008,  0.0900791 , -0.33690856],
        [-0.4       , -0.04649906,  0.10182854,  0.90965311]]),
 array([[ -5.        ,  -9.8       , -13.6       ,  -6.8       ,
          -5.6       ],
        [  0.        ,  -1.72046505,  -4.48715886,  -3.11543672,
          -3.55717775],
        [  0.        ,   0.        ,   3.45042105,   3.05485632,
          -0.32506804],
        [  0.        ,   0.        ,   0.        ,  -2.39205078,
           5.08731926]]))

In [ ]:
# Yes, we could notice the difference, this is beceause of the "nan" character. 
# It arrives when at the very last iteration, we reach at a scalar x and try to perform Householder reflectors 
# and y becomes equals to x and thus v becomes 0 and making inner(v,v)=0 and 
# thereby making h=i-2*np.outer(v, v)/ np.inner(v,v) a "nan" character.

# (4) Additional Task 1

You can work the additional tasks in this section if you are already done with the main part of the problem sheet.

We first want to take a look at Hessenberg matrices, which often appear in numerical linear algebra, e.g., for the approximation of eigenvalues, matrix functions, or the solution of linear systems (which is are powerful tools to tackle many data science problems). A Hessenberg matrix $H_m\in\mathbb{R}^{m \times m}$ is defined by possessing the following sparsity structure:

$$
H_m = \begin{bmatrix}
\ast & \ast & \cdots & \ast & \ast\\
\ast & \ast & \cdots & \ast & \ast\\
 & \ast & \ddots & \vdots & \vdots\\
 & & \ddots & \ast & \ast\\
0 & & & \ast & \ast
\end{bmatrix}
$$

We now want to compute the QR decomposition of $H_m$. Of course, your functions from the previous tasks (as well as the second additional task) should work here, but we want to find an alternative way to tackle this problem.

**Task: Create a Hessenberg matrix $H_m$ and write a function to compute its QR decomposition WITHOUT using Householder reflections. The input should be $H_m$ and the output the two matrices $Q$ and $R$.**